In [4]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

# -----------------------------
# Load analytics data
# -----------------------------
df = pd.read_csv("sales_revenue_analytics.csv")
df["order_delivered_customer_date"] = pd.to_datetime(
    df["order_delivered_customer_date"]
)

In [13]:
monthly_revenue = (
    df.groupby(
        pd.Grouper(
            key="order_delivered_customer_date",
            freq="ME"
        )
    )["total_item_revenue"]
    .sum()
)
monthly_revenue = monthly_revenue.iloc[:-1]

print("Monthly revenue used for forecasting:")
print(monthly_revenue.tail())
print(monthly_revenue.describe())

Monthly revenue used for forecasting:
order_delivered_customer_date
2018-05-31    1170436.47
2018-06-30    1171020.32
2018-07-31     947840.37
2018-08-31    1347294.08
2018-09-30      12875.18
Freq: ME, Name: total_item_revenue, dtype: float64
count    2.400000e+01
mean     6.424186e+05
std      4.414117e+05
min      9.608500e+02
25%      2.877826e+05
50%      6.486721e+05
75%      1.005795e+06
max      1.347294e+06
Name: total_item_revenue, dtype: float64


In [14]:
# Use last 3 months as baseline
baseline_revenue = monthly_revenue.tail(3).mean()

print("Baseline monthly revenue:", baseline_revenue)

Baseline monthly revenue: 769336.5433333335


In [15]:
forecast_periods = 6
monthly_growth_rate = 0.03   # 3% month-over-month (reasonable)

forecast_values = []

current_value = baseline_revenue

for i in range(forecast_periods):
    current_value = current_value * (1 + monthly_growth_rate)
    forecast_values.append(current_value)


In [16]:
forecast_dates = pd.date_range(
    start=monthly_revenue.index[-1] + pd.offsets.MonthEnd(1),
    periods=forecast_periods,
    freq="ME"
)

forecast_df = pd.DataFrame({
    "date": forecast_dates,
    "forecast_revenue": forecast_values
})

print(forecast_df)


        date  forecast_revenue
0 2018-10-31     792416.639633
1 2018-11-30     816189.138822
2 2018-12-31     840674.812987
3 2019-01-31     865895.057377
4 2019-02-28     891871.909098
5 2019-03-31     918628.066371


In [18]:
forecast_df.to_csv(
    "monthly_revenue_forecast.csv",
    index=False
)
